This script is created to get the list of devices from eUDT site. It uses eUDT API with an authorization token
received from UDT Programming Specialist. User has to authenticate authorization token, then use received
access_token to get information from eUDT site. Token is valid for 5 minutes, after that it has to be refreshed.
Refreshing token is valid for 30 minutes, after that whole authorization process has to be done again.

In [ ]:
# import the libraries
import json
import requests
import pprint
import pandas as pd

In [ ]:
def authentication(headers, params, api_url):
    """
    This code is used to authenticate my API Token and receive an access token together with refresh token. Access token is valid for 5 minutes,
    after that it has to be refreshed with usage of refresh token which is valid 30 minutes. Authentication allows user to use eUdt API.
    """
    response = requests.post(api_url, headers = headers, json=params)

    access_token = response.json()["accessToken"]
    authorization = f'Bearer {access_token}'

    refresh_token = response.json()["refreshToken"]

    return authorization, refresh_token

In [ ]:
def refresh_the_token(api_url, refresh_token, token):
    """
    This code is used to refresh access_token. It is valid for 30 minutes, after that main token has to be authenticated again
    """
    headers = {
    "Content-Type": "application/json",
    "X-Api-Token": token,
    }
    params = {
        "refreshToken" : refresh_token
    }

    response = requests.put(api_url, headers = headers, json=params)

    access_token = response.json()['accessToken']
    authorization = f'Bearer {access_token}'

    return authorization

In [ ]:
def logging():
    """
    This code serves for keeping the logging credentials and headers in one place.
    """
    # Opening JSON file
    f = open('udt.json')  # d.paw email   

    # returns JSON object as a dictionary
    data = json.load(f)

    # login credentials
    email = data['email']
    password = data['password']
    token = data['token']

    headers = {
    "Content-Type": "application/json",
    "X-Api-Token": token,
    }

    params = {
    "login": email,
    "password": password
    }

    return headers, params, token

In [ ]:
def get_access(api_url):
    """
    This code is created to get access to eUDT api.
    """
    headers, params, token = logging()

    authorization, refresh_token = authentication(headers, params, api_url)

    return authorization, refresh_token, token

In [ ]:
def open_site(authorization, id=False):
    """
    This code is created to take information about the devices from eUDT site.
    """
    if id:
        id = "/" + id
    else:
        id = ""
        
    payload = ""

    headers = {
    "authority": "api.eudt.gov.pl",
    "accept": "application/json, text/plain, */*",
    "accept-language": "pl,en;q=0.9,en-GB;q=0.8,en-US;q=0.7",
    "cache-control": "no-cache",
    "cookie": "",
    "expires": "0",
    "origin": "https://eudt.gov.pl",
    "pragma": "no-cache",
    "sec-ch-ua": "^\^Chromium^^;v=^\^112^^, ^\^Microsoft",
    "sec-ch-ua-mobile": "?0",
    "sec-ch-ua-platform": "^\^Windows^^",
    "sec-fetch-dest": "empty",
    "sec-fetch-mode": "cors",
    "sec-fetch-site": "same-site",
    "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36",
    "x-context": "32283",
    "x-xsrf-token": "YmRlM2ZmNThiMGVlYmE2NmUxN2FjNDZlZjkzMmQxOGQyMDk5MGQ0NjdjNjNmOGJjOTJiMTI4MWMxMmJlNzEwODthOGM1YzZlOS0xNTVjLTRmMTQtOTc4Ni1jNDcwYTVjMTg3ZDc=",
    "Authorization" : authorization
    }

    # getting applications
    url = "https://api.eudt.gov.pl/Application" + id
    # url = "https://api.eudt.gov.pl/Device"
    # print("tu jestem")

    response = requests.request("GET", url, data=payload, headers=headers)
    # print("kolejny krok")

    # pprint.pprint(response.json()['number'])

    # return response.json()['data']
    return response.json()

In [ ]:
def get_numbers(ids_data, authorization, api_url, refresh_token, token):
    list_of_ids = {}
    for i in ids_data:
        id = str(i['id'])
        try:
            number = open_site(authorization, id)
        except:
            print("refreshing for numbers")
            authorization = refresh_the_token(api_url, refresh_token, token)
            print("refreshed")
            number = open_site(authorization, id)

        list_of_ids[id] = number
           
    return list_of_ids

In [ ]:
def get_applications_data(api_url, refresh_token, token):
    """
    This is the main function which serves to get all the data about the devices that user has access to.
    It takes just 3 positional arguments, api_url is predefined, refresh_token is taken from get_access function,
    token is given from eUDT.
    """
    # take data from eUDT site. Try to get it directly, if not available, then refresh the token, if token is not 
    # valid anymore, authorize it again
    try:
        json_data = open_site(authorization)

    except:
        try:
            print("refreshing token")
            authorization = refresh_the_token(api_url, refresh_token, token)
            json_data = open_site(authorization)

        except:
            print("get the new token")
            authorization, refresh_token, token = get_access(api_url)
            json_data = open_site(authorization)
    
    ids_data = json_data['data']

    list_of_ids = get_numbers(ids_data, authorization, api_url, refresh_token, token)
    print(list_of_ids)
   
    # create empty dataframe for keeping data about the devices that the user has access to
    df = pd.DataFrame(columns=["ID", "Number"], index=range(0))

    # loop through every device to get the details about it
    for i, (key, value) in enumerate(list_of_ids.items()):
        df.loc[i, 'ID'] = key
        df.loc[i, 'Number'] = value

    # export excel file with all the data
    df.to_excel("List of Applications from UDT.xlsx", index=False)

In [ ]:
api_url = "https://api.eudt.gov.pl/token"
authorization, refresh_token, token = get_access(api_url)

In [ ]:
get_applications_data(api_url, refresh_token, token)